## Работа с составными типами данных

При обработке массивов чисел есть несколько подходов.

Рассмострим несколько типов составных переменных и взаимосвязи между ними.

В качестве данных будем использовать текстовый файл с данными по времени реакции (reaction time, RT).

In [ ]:
u='rt.tsv'
u

## 1. Список
Входит в базовый комплект `Python` и не требует импорта библиотек.

Достоинство - универсальность, недостатки - требует перебора входящих элементов, ибо никак не предполагает их единообразия: в одном списке могут быть и числа, и строки, и сложные объекты.

In [ ]:
#создаем пустой список
vv=[]

#открываем файл
with open(u) as f:
    #пропускаем строку заголовков
    print('Строка заголовков:' + f.readline())
    # из каждой строчки извлекаем число между 1-м и 2-м пробелами или табуляциями (2-ю колонку)
    for l in f.readlines():
#       print(v)  # выводит на экран каждую строку, если разкомментировать
        v = l.split()[1]
        vv.append(float(v))
        
vv        

У списка есть только длина, но в его состав могут входить другие списки.

In [ ]:
len(vv)

Сгруппируем список из 6 списочков по 8 значений в каждом.

In [ ]:
vv6x8 = []
i = 0 #текущая позиция
for igroup in range(6):
    v1 = vv[i:i+8]
    vv6x8.append(v1)
    i += 8 #сдвигаемся к началу следующей группы
        
vv6x8

Необходимо правильно рассчитывать размеры, и проверять крайние группы, чтобы при указании индексов не выйти за пределы массива.

Из функций обработки в чистом `Python` есть только сумма. Вот так можно посчитать средние значения для каждой группы.

In [ ]:
[sum(gr)/len(gr) for gr in vv6x8]

## 2. Numpy Array

Массивы чисел, с которыми можно обращаться как с единым целым. Библиотека `numpy` включает функции линейной алгебры для работы с матрицами.

Типичный импорт в виде `np`. В блокноте более удобным является импорт командой `%pylab` (см. в других примерах).

In [ ]:
import numpy as np
np.set_printoptions(suppress=True)  #не использовать научную нотацию

В `numpy` есть функция загрузки таблиц с числами `loadtxt`. Поскольку в нашем файле есть заголовки (строки, а не числа) и колонки с нечисловыми значениями, то указываем дополнительные параметры: пропустить 1 ряд, использовать 2-ю колонку.

In [ ]:
x = np.loadtxt(u, skiprows=1, usecols=(1,))
x

In [ ]:
x.shape

Массивы чисел в `numpy` могут иметь много измерений. Чтобы изменить форму массива, достаточно указать новые размеры по нужным измерениям.

In [ ]:
X = x.reshape(-1,8)
X

In [ ]:
X.shape

In [ ]:
type(X)

Мы получили шесть групп значений, но в двух группах есть пропуски, помеченные `nan`. Чтобы пустые значения не мешали в вычислениях, можно сделать маску, которая исключит помеченные значения из вычислений.

In [ ]:
Xmask = np.ma.masked_array(X, np.isnan(X))
type(Xmask)

С массивом можно обращаться к с единым целым. При этом можно применять разные операции к колонкам и строчкам. В случае необходимости менять форму и переворачивать (транспонировать).

Есть методы для базовой статистики.

In [ ]:
X.mean()

In [ ]:
X.mean(axis=1)

In [ ]:
Xmask.mean(axis=1)

In [ ]:
Xmask.mean(axis=1).round(2)

Наличие пустых значений - типичная ситуация при анализе данных, поэтому созданы специальные функции для работы с `nan`.

In [ ]:
np.nanmean(X, axis=1).round(2)

In [ ]:
np.nanmean(X.T, axis=0).round(2)

Свойство `.T` (T большое) - транспонирование.

Очень удобно выполнять действия с несколькими массивами. При этом автоматически подбираются соответствия между количеством значений в каждом.

Распределенное соответствие

![](numpy_broadcasting.png)

Например, нормализуем массив - вычтем среднее и разделим на стандартное отклонение каждую строчку.

In [ ]:
Xnorm = np.array((Xmask.T - Xmask.T.mean(axis=0))/Xmask.T.std(axis=0)).T.round(3)
Xnorm

Массив можно редуцировать до списка.

In [ ]:
Xnorm.tolist()

In [ ]:
type(_)

##  3. Pandas DataFrame

Для работы со структурированными данными библиотека `pandas` предлагает структуры:
- 1-мерная Series
- 2-мерная DataFrame
- 3-мерная Panel

Типичный импорт в виде `pd`. Библиотека включает массу функций для удобного обращения с файлами, датами, статистикой и рисунками.

In [ ]:
import pandas as pd

In [ ]:
D = pd.read_table(u)
D

При чтении таблицы создаются индексы - для наблюдений (строчек) и для переменных (колонок).

In [ ]:
D.columns

In [ ]:
D.index

Можно задать в качестве индекса одну из колонок. По-умолчанию используется автоматическая нумерация (как в этом примере).

Каждая строчка или колонка `DataFrame` - это серия с тем же индексом. К значениям можно обращаться по их индексу в любом порядке. Например, чтобы получить третье по счету значение ВР:

In [ ]:
D['v'][2]  # вначале колонка, потом позиция в колонке

In [ ]:
D.loc[2]['v']  # вначале строка, потом колонка

In [ ]:
D.loc[2,'v']  # обе координаты строка и колонка

Способ с одновременным указанием строки и колонки является предпочтительным, потому что в одно действие происходит обращение к нужной ячейке таблицы. 
Именно этот способ используют для замены конкретного значения, например
```py
D.loc[2,'v'] = D['v'].mean()
```

Можно использовать индекс позиции, как в массивах numpy.

In [ ]:
D.iloc[2,1]

Таблицы включают мощные средства манипуляции данными.

При необходимости разбиения массива данных на группы признаки принадлежности к разным группам добавляют в виде новых колонок, а уже информацию в этих колонках используют в разных сочетаниях при отборе подвыборок и сравнительном анализе.

Добавим в качестве группового признака номер группы для разделения всей последовательности на 6 одинаковых групп, идущих друг за другом.

In [ ]:
ngroup = 6
D['g'] = np.kron(range(1,ngroup+1), np.ones(int(len(D)/ngroup), int))
D

Например, чтобы посчитать среднее ВР для разных МСИ (в колонке `g`), можно сделать так:

In [ ]:
D.groupby('g')['v'].mean()

Таблицы и Серии подходят для использования в операциях линейной алгебры из `numpy`. Если надо,  значения таблицы можно извлечь в виде массива или в виде списка.

In [ ]:
D.values

И, наоборот, из массива можно сделать датафрейм (таблицу). Если не указать индексы - они создадутся автоматически.

In [ ]:
pd.DataFrame(X)

В конце посмотрим на типы переменных, созданных разными способами.

In [ ]:
type(vv), type(X), type(D['g']), type(D)